In [2]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


In [3]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)


In [13]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=np.mean(train_x), variance=np.var(train_x)),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            epochs=epochs,
            validation_data=(valid_x, valid_y),
            batch_size=64,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    patience=3, restore_best_weights=True
                )
            ],
        )

        # Evaluate model on the test data subset
        eval_result = model.evaluate(test_x, test_y)
        eval_rmse = eval_result[1]

        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": eval_rmse})

        # Log the model within the MLflow run
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"eval_rmse": eval_rmse, "status": STATUS_OK, "model": model, "loss": eval_rmse}


In [6]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result


In [7]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}


In [14]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["eval_rmse"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['eval_rmse']}")


  0%|          | 0/8 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

  0%|          | 0/8 [00:00<?, ?trial/s, best loss=?]

2024-01-11 11:15:33.370788: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node SGD/AssignVariableOp.


46/46 [==============================] - 1s 22ms/step - loss: 6.6736 - root_mean_squared_error: 2.5833 - val_loss: 3.2331 - val_root_mean_squared_error: 1.7981

Epoch 2/3                                            

46/46 [==============================] - 0s 7ms/step - loss: 2.8383 - root_mean_squared_error: 1.6847 - val_loss: 2.7255 - val_root_mean_squared_error: 1.6509

Epoch 3/3                                            

46/46 [==============================] - 0s 7ms/step - loss: 2.3988 - root_mean_squared_error: 1.5488 - val_loss: 2.3065 - val_root_mean_squared_error: 1.5187

39/39 [==============================] - 0s 5ms/step - loss: 2.4303 - root_mean_squared_error: 1.5589

  0%|          | 0/8 [00:02<?, ?trial/s, best loss=?]INFO:tensorflow:Assets written to: /var/folders/ld/8bgljzsn4sx_x1lb3wl_mnnm0000gn/T/tmpjwnh1xst/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/ld/8bgljzsn4sx_x1lb3wl_mnnm0000gn/T/tmpjwnh1xst/model/data/model/assets



...

Epoch 1/3                                                                      

46/46 [==============================] - 1s 11ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan

Epoch 2/3                                                                      

46/46 [==============================] - 0s 6ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan

Epoch 3/3                                                                      

46/46 [==============================] - 0s 7ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan

39/39 [==============================] - 0s 5ms/step - loss: nan - root_mean_squared_error: nan

 88%|████████▊ | 7/8 [00:36<00:04,  4.94s/trial, best loss: 0.9118534326553345]INFO:tensorflow:Assets written to: /var/folders/ld/8bgljzsn4sx_x1lb3wl_mnnm0000gn/T/tmp3n6z1kvn/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/ld/8bgljzsn4sx_x1lb3wl_mnnm0000gn/T/tmp3n6z1kvn/model/data/model/assets



100%|██████████| 8/8 [00:40<00:00,  5.01s/trial, best loss: 0.9118534326553345]
INFO:tensorflow:Assets written to: /var/folders/ld/8bgljzsn4sx_x1lb3wl_mnnm0000gn/T/tmpkztpu109/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/ld/8bgljzsn4sx_x1lb3wl_mnnm0000gn/T/tmpkztpu109/model/data/model/assets



Best parameters: {'lr': 0.0031683216722915046, 'momentum': 0.712097687201572}
Best eval rmse: 0.9118534326553345
